In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_full = pd.read_csv("./train.csv", index_col="Id")
X_test_full = pd.read_csv("./test.csv", index_col="Id")

X_full.dropna(axis=0, subset=["SalePrice"], inplace=True)

y = X_full.SalePrice
X_full.drop(["SalePrice"], axis=1, inplace=True)
X = X_full.select_dtypes(exclude=["object"])
X_test = X_test_full.select_dtypes(exclude=["object"])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [10]:
print(X_train.shape)

(1168, 36)


In [13]:
missing_values_count_by_column = (X_train.isnull().sum())
print(missing_values_count_by_column[missing_values_count_by_column > 0].sum())
print(missing_values_count_by_column[missing_values_count_by_column > 0])


276
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return(mean_absolute_error(y_test, predictions))


In [22]:
cols_with_missing = [col for col in X_train if X_train[col].isnull().any()]
print(cols_with_missing)
X_train_dropped = X_train.drop(cols_with_missing, axis=1)
X_valid_dropped = X_valid.drop(cols_with_missing, axis=1)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


In [23]:
print(score_dataset(X_train_dropped, X_valid_dropped, y_train, y_valid))

17837.82570776256


In [25]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

18062.894611872147


In [30]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
iterative_imputer = IterativeImputer(initial_strategy="median")

final_X_train = pd.DataFrame(iterative_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(iterative_imputer.transform(X_valid))

final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [33]:
model = RandomForestRegressor(n_estimators=700, random_state=1)
model.fit(final_X_train, y_train)

RandomForestRegressor(n_estimators=700, random_state=1)

In [34]:
predictions = model.predict(X_valid)
mea = mean_absolute_error(y_valid, predictions)
print(mea)

17608.691578604044


In [36]:
imputed_X_test = pd.DataFrame(iterative_imputer.fit_transform(X_test))
imputed_X_test.columns = X_test.columns

In [37]:
predictions2 = model.predict(imputed_X_test)

In [41]:
print(predictions2[:5])

[128300.97428571 156492.67428571 184184.67857143 184017.83857143
 196985.20714286]


In [43]:
output = pd.DataFrame({
    "Id": imputed_X_test.index,
    "SalePrice": predictions2
})


In [45]:
output.to_csv("submission2.csv")

In [49]:
output.iloc[5:10]

,Id,SalePrice
5,5,183520.122857
6,6,168497.567143
7,7,174146.467143
8,8,182880.927143
9,9,124220.075714
